info:
- plus (L) strand
- minus (H) strand 
- positions in the abasic sites table are 1-based (there are position 16299 in the table)

In [1]:
from collections import defaultdict

import ete3
from ete3 import PhyloTree
from Bio import SeqIO

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 64)

In [2]:
PATH_TO_DATA = './data/'
path_to_refseq = PATH_TO_DATA + 'mm10_ChrM_oneline.fasta'
path_to_genbank = PATH_TO_DATA + 'NC_005089.1.gb'
path_to_abasic_data = PATH_TO_DATA + '41467_2022_33594_MOESM26_ESM.csv'

In [3]:
refseq = str(next(SeqIO.parse(path_to_refseq, 'fasta')).seq)
genbank = next(SeqIO.parse(path_to_genbank, 'genbank'))

assert genbank.seq == refseq

In [4]:
# BED format: 0-based positions, Start included, End excluded
abasic = pd.read_csv(path_to_abasic_data, sep=';')
abasic.head()

,Chr(mm10),Start(mm10),End(mm10),NumberOfReads,Strand
0,chrM,68,69,1,+
1,chrM,69,70,2,+
2,chrM,70,71,1,+
3,chrM,74,75,4,+
4,chrM,81,82,2,+


In [5]:
genbank.features[-6:]

[SeqFeature(FeatureLocation(ExactPosition(15422), ExactPosition(16299), strand=1), type='D-loop'),
 SeqFeature(FeatureLocation(ExactPosition(15450), ExactPosition(15509), strand=1), type='misc_feature'),
 SeqFeature(FeatureLocation(ExactPosition(15514), ExactPosition(15558), strand=1), type='misc_feature'),
 SeqFeature(FeatureLocation(ExactPosition(16034), ExactPosition(16058), strand=1), type='misc_feature'),
 SeqFeature(FeatureLocation(ExactPosition(16088), ExactPosition(16104), strand=1), type='misc_feature'),
 SeqFeature(FeatureLocation(ExactPosition(16113), ExactPosition(16131), strand=1), type='misc_feature')]

In [6]:
DLOOP_START = 15422 # 0-based

In [7]:
# 0-based
# collect triplets on mm10

df = pd.DataFrame([[i, x] for i, x in enumerate(refseq, 0)], columns=['site', 'nuc'])
df['down'] = [''] + list(df.nuc.values[:-1])
df['up'] = list(df.nuc.values[1:]) + ['']
df['triplet'] = df['down'].str.lower() + df['nuc'] + df['up'].str.lower()
df = df[(df.triplet.str.len() == 3) & (df.site < DLOOP_START)]
df[['site', 'down', 'nuc', 'up', 'triplet']]

,site,down,nuc,up,triplet
1,1,G,T,T,gTt
2,2,T,T,A,tTa
3,3,T,A,A,tAa
4,4,A,A,T,aAt
5,5,A,T,G,aTg
...,...,...,...,...,...
15417,15417,T,T,C,tTc
15418,15418,T,C,T,tCt
15419,15419,C,T,T,cTt
15420,15420,T,T,G,tTg


In [8]:
# merge AP sites data with mm10 triplets (site-specific)
merged = df.merge(abasic, right_on='Start(mm10)', left_on='site')

In [9]:
merged.Strand.value_counts()

-    8422
+    5919
Name: Strand, dtype: int64

In [10]:
merged

,site,nuc,down,up,triplet,Chr(mm10),Start(mm10),End(mm10),NumberOfReads,Strand
0,2,T,T,A,tTa,chrM,2,3,2,-
1,3,A,T,A,tAa,chrM,3,4,1,-
2,4,A,A,T,aAt,chrM,4,5,2,-
3,5,T,A,G,aTg,chrM,5,6,4,-
4,6,G,T,T,tGt,chrM,6,7,2,-
...,...,...,...,...,...,...,...,...,...,...
14336,15408,T,T,A,tTa,chrM,15408,15409,1,-
14337,15409,A,T,A,tAa,chrM,15409,15410,2,-
14338,15410,A,A,A,aAa,chrM,15410,15411,2,-
14339,15411,A,A,C,aAc,chrM,15411,15412,1,-


In [11]:
# calculate coverage af AP sites on triplets, i.e. AP sites frequency for each triplet
coverage = merged.groupby(['Strand', 'triplet']).NumberOfReads.sum().reset_index()
coverage

,Strand,triplet,NumberOfReads
0,+,aAa,510
1,+,aAc,327
2,+,aAg,224
3,+,aAt,370
4,+,aCa,257
...,...,...,...
123,-,tGt,136
124,-,tTa,1581
125,-,tTc,3308
126,-,tTg,1710


In [12]:
# calculate trinucleotide counts on genome (like expected values)
triplet_counts = defaultdict(int)
for i in range(1, len(refseq)-1):
    tri = refseq[i-1: i+2].lower()
    tri = tri[0] + tri[1].upper() + tri[2]
    triplet_counts[tri] += 1

len(triplet_counts)

64

In [13]:
triplet_counts_df = pd.Series(triplet_counts).rename('count').to_frame()
triplet_counts_df.index.rename('triplet', inplace=True)
triplet_counts_df = triplet_counts_df.reset_index()
triplet_counts_df.head()

,triplet,count
0,gTt,123
1,tTa,506
2,tAa,560
3,aAt,574
4,aTg,196


In [15]:
# need to get reverse complements of H-strand ('-')
transcriptor = str.maketrans("ACGTacgt", "TGCAtgca")


def rev_comp(mut: str):
    new_mut = mut[-1] + mut[1:-1] + mut[0]
    new_mut = new_mut.translate(transcriptor)
    return new_mut

rev_comp('aCg')

'cGt'

In [16]:
ap_cov = coverage.merge(triplet_counts_df, on='triplet')
ap_cov['triplet_true'] = ap_cov['triplet']
ap_cov.loc[ap_cov['Strand'] == '-', 'triplet_true'] = \
    ap_cov.loc[ap_cov['Strand'] == '-', 'triplet_true'].apply(rev_comp)
ap_cov['APcount_norm'] = ap_cov['NumberOfReads'] / ap_cov['count']
ap_cov

,Strand,triplet,NumberOfReads,count,triplet_true,APcount_norm
0,+,aAa,510,642,aAa,0.794393
1,-,aAa,661,642,tTt,1.029595
2,+,aAc,327,485,aAc,0.674227
3,-,aAc,2002,485,gTt,4.127835
4,+,aAg,224,211,aAg,1.061611
...,...,...,...,...,...,...
123,-,tTc,3308,367,gAa,9.013624
124,+,tTg,156,146,tTg,1.068493
125,-,tTg,1710,146,cAa,11.712329
126,+,tTt,196,356,tTt,0.550562


In [17]:
# final table with AP sites adjusted counts
# '-' strand is H
# '+' strand is L
results = ap_cov.pivot('triplet_true', 'Strand', 'APcount_norm')
results.sort_values('-')

Strand,+,-
triplet_true,,
aCt,0.547558,0.560241
aCc,0.487324,0.673684
aTt,0.507018,0.675958
aTa,0.446640,0.758893
aTc,0.451187,0.816993
aTg,0.688776,0.889952
aAc,0.674227,0.894309
aAt,0.644599,1.001754
tTt,0.550562,1.029595


## check with modified approach by VSh (scripts)

In [18]:
vsh_results = pd.read_csv('../data/AbasicSitesMtDNAcontext_HLcompare.csv', sep=';')
vsh_results

,tripletH,origtriH,countH,deepcountH,avgH,tripletL,countL,deepcountL,avgL
0,tTt,aAa,642,661,1.029595,tTt,356,196,0.550562
1,aTt,aAt,574,388,0.675958,aTt,570,289,0.507018
2,cTt,aAg,211,602,2.853081,cTt,326,429,1.315951
3,gTt,aAc,485,2002,4.127835,gTt,123,303,2.463415
4,tAt,aTa,506,1375,2.717391,tAt,506,756,1.494071
5,aAt,aTt,570,571,1.001754,aAt,574,370,0.644599
6,cAt,aTg,196,1312,6.693878,cAt,418,1484,3.550239
7,gAt,aTc,379,2034,5.366755,gAt,153,586,3.830065
8,tCt,aGa,201,376,1.870647,tCt,326,318,0.975460
9,aCt,aGt,166,93,0.560241,aCt,389,213,0.547558


In [19]:
vsh_results = vsh_results.merge(results, left_on='tripletH', right_index=True)

assert (vsh_results.avgH - vsh_results['-']).max().round(2) == 0
assert (vsh_results.avgL - vsh_results['+']).max().round(2) == 0